In [1]:
#Code Reference : https://qiskit.org/documentation/_modules/qiskit/finance/applications/ising/portfolio.html#get_operator
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime

import gurobipy as gp
from gurobipy import GRB
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.8/site-packages/qiskit/finance/__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')
/opt/anaconda3/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


In [3]:
#importing data.csv file as a dataframe 
df= pd.read_csv('/Users/ashwinsaxena/Google Drive/QF/PortfolioOptimisation/data.csv',parse_dates=True,index_col="Date1")

In [4]:
# creating data frame with chosen columns
df1= df[['Amazon','AMD','Apple','Cisco','FB','Microsoft','Qualcomm','Starbuck','Tesla','ZNGA']]

In [5]:
#Checking the dataframe
df1.head()

,Amazon,AMD,Apple,Cisco,FB,Microsoft,Qualcomm,Starbuck,Tesla,ZNGA
Date1,,,,,,,,,,
2021-06-09,3281.15,79.96,127.13,54.02,330.25,253.59,132.89,111.39,598.78,10.31
2021-06-08,3264.11,80.89,126.74,54.13,333.68,252.57,134.20,111.52,603.59,10.40
2021-06-07,3198.01,81.35,125.90,53.92,336.58,253.81,133.32,111.33,605.13,10.36
2021-06-04,3206.22,81.58,125.89,54.07,330.35,250.79,134.34,111.99,599.05,10.49
2021-06-03,3187.01,80.28,123.54,53.33,326.04,245.71,131.78,111.12,572.84,10.40


In [6]:
# indexing the column names as stocks
stocks = df1.columns


In [7]:
#Calculating the mean vector
mu = df1.mean()

In [8]:
mu

Amazon       3164.115159
AMD            79.976488
Apple         119.126835
Cisco          45.563313
FB            274.135813
Microsoft     223.655873
Qualcomm      128.348532
Starbuck       96.006905
Tesla         538.900278
ZNGA            9.744048
dtype: float64

In [9]:
#saving mean as an array
mu=np.array([3164.115159,79.976488, 119.126835,45.563313,274.135813,223.655873,128.348532,96.006905,538.900278,9.744048])

In [42]:
mu

array([3164.115159,   79.976488,  119.126835,   45.563313,  274.135813,
        223.655873,  128.348532,   96.006905,  538.900278,    9.744048])

In [43]:
#saving the covariance of the stock data
stock_cov=df1.cov()

In [44]:
#saving the matrix as an array
sigma = np.array(stock_cov)

In [45]:
sigma

array([[ 2.97241526e+04,  1.29303795e+03,  1.81136808e+03,
         8.96843097e+01,  2.88181476e+03,  1.96537733e+03,
         1.97697322e+03,  1.14329089e+03,  1.77623797e+04,
         4.18366308e+01],
       [ 1.29303795e+03,  1.24818517e+02,  1.24717319e+02,
        -5.39243931e+00,  1.19672278e+02,  7.29481056e+01,
         1.93500336e+02,  8.39716170e+01,  1.47653091e+03,
         7.52889172e-01],
       [ 1.81136808e+03,  1.24717319e+02,  1.77846272e+02,
         1.58751836e+01,  2.31571266e+02,  1.79034355e+02,
         2.37118418e+02,  1.48604251e+02,  2.23512380e+03,
         4.53805405e+00],
       [ 8.96843097e+01, -5.39243931e+00,  1.58751836e+01,
         2.16347725e+01,  5.83537597e+01,  5.94520801e+01,
         8.71594373e+00,  3.65333728e+01,  3.40864368e+02,
         2.62951741e+00],
       [ 2.88181476e+03,  1.19672278e+02,  2.31571266e+02,
         5.83537597e+01,  6.57773089e+02,  3.69248854e+02,
         2.61946837e+02,  2.67884695e+02,  2.40778239e+03,
         6.

In [46]:
# qiakit library

num_assets=10
q = 0.5                   # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets 
qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

In [47]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [48]:
#classical simulation
exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
result = exact_eigensolver.run()

print_result(result)

Optimal: selection [0 1 0 1 0 0 0 0 0 1], value -23.6898

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 1 0 1 0 0 0 0 0 1]	-23.6898		1.0000
[1 1 1 1 1 1 1 1 1 1]	76061.1110		0.0000
[1 1 1 1 0 0 1 0 1 0]	60049.6896		0.0000
[0 0 1 1 1 0 1 0 1 0]	26650.9013		0.0000
[1 1 0 1 1 0 1 0 1 0]	61440.0113		0.0000
[0 1 0 1 1 0 1 0 1 0]	25728.1602		0.0000
[1 0 0 1 1 0 1 0 1 0]	58370.2295		0.0000
[0 0 0 1 1 0 1 0 1 0]	23971.4163		0.0000
[1 1 1 0 1 0 1 0 1 0]	65592.2264		0.0000
[0 1 1 0 1 0 1 0 1 0]	28158.6916		0.0000
[1 0 1 0 1 0 1 0 1 0]	62392.3349		0.0000
[0 0 1 0 1 0 1 0 1 0]	26271.8379		0.0000
[1 1 0 0 1 0 1 0 1 0]	60972.5313		0.0000
[0 1 0 0 1 0 1 0 1 0]	25370.3645		0.0000
[1 0 0 0 1 0 1 0 1 0]	57917.3570		0.0000
[0 0 0 0 1 0 1 0 1 0]	23628.2282		0.0000
[0 1 1 1 0 0 1 0 1 0]	25408.2852		0.0000
[0 1 1 1 1 0 1 0 1 0]	28552.3624		0.0000
[1 0 1 1 0 0 1 0 1 0]	56974.8628		0.0000
[0 0 1 1 0 0 1 0 1 0]	23646.4963

In [49]:
# quantum vqe simulation

backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

print_result(result)

/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.optimizers',
/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/algorithms/vq_algorithm.py:70: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refactored to qiskit.algorithms.VariationalAlgorithm (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.algorithms.VQAlgorithm',
/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit

Optimal: selection [0. 0. 1. 0. 0. 0. 0. 1. 0. 1.], value 158.9031

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 1 0 0 0 0 1 0 1]	158.9031		0.2911
[0 0 1 0 0 0 0 0 0 1]	54.9678		0.2467
[0 1 0 0 0 0 0 1 0 1]	103.1217		0.1067
[0 1 1 0 0 0 0 0 0 1]	112.8708		0.0774
[0 0 0 0 0 0 0 0 0 1]	150.6334		0.0714
[0 0 1 0 1 0 0 0 0 1]	297.7946		0.0245
[0 0 1 0 0 0 0 0 0 0]	129.7963		0.0188
[0 0 0 0 1 0 0 1 0 1]	365.1047		0.0185
[0 1 1 0 0 0 0 1 0 1]	320.7777		0.0163
[0 1 0 0 0 1 0 1 0 1]	307.4255		0.0139
[0 1 1 0 1 0 0 1 0 1]	991.1615		0.0113
[0 0 0 0 0 1 0 1 0 1]	197.3202		0.0078
[0 1 0 0 1 0 0 1 0 1]	521.9343		0.0072
[0 1 0 0 0 0 0 0 0 0]	142.4328		0.0063
[0 1 0 0 0 0 0 0 0 1]	63.8191		0.0057
[0 0 1 0 1 0 0 1 0 1]	689.6146		0.0055
[0 0 1 1 0 0 0 1 0 0]	175.3631		0.0044
[0 1 0 0 0 0 0 1 0 0]	155.7043		0.0039
[0 1 0 0 1 0 0 0 0 1]	194.7469		0.0038
[0 1 1 0 1 0 0 0 0 1]	495.3699		0.0036
[0 1 0 1 0 0 0 1 0 0]